In [21]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Read the CSV file
data = pd.read_csv('ndx_daily.csv')
data['Date'] = pd.to_datetime(data['Date'])

# Calculate the earnings
initial_investment = 100
leverage = 2
investment_value = initial_investment # * leverage

# Define start and end dates
start_date = '2020-01-01'
end_date = '2021-01-01'  # For example

# Filter data based on start and end dates
filtered_data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]

# Initialize figure with subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Investment Value Over Time", "Open Price"))

# Plot investment value
dates = []
values = []
hover_text = []
hover_text_open = []

# Iterate over each row in the filtered DataFrame
for index, row in filtered_data.iterrows():
    open_price = row['Open']
    if index == filtered_data.index[0]:
        continue  # Skip the first row since there's no previous price to calculate change from

    dates.append(row['Date'])
    values.append(investment_value)
    
    previous_open_price = filtered_data.loc[index - 1, 'Open']
    daily_change = (open_price - previous_open_price) / previous_open_price
    daily_return = 1 + daily_change * leverage
    investment_value *= daily_return

    hover_text.append(f"Date: {row['Date']}<br>"
                      f"Open: {open_price:.2f}<br>"
                      f"Daily Change: {daily_change:.2%}<br>"
                      f"Investment Value: {investment_value:.2f}")
    
    hover_text_open.append(f"Date: {row['Date']}<br>"
                      f"Open: {open_price:.2f}<br>"
                      f"Daily Change: {daily_change:.2%}<br>")

# Add trace for investment value
fig.add_trace(go.Scatter(x=dates, y=values, mode='lines+markers', name='Investment Value',
                         hovertext=hover_text, hoverinfo='text'), row=1, col=1)

# Add trace for open price
fig.add_trace(go.Scatter(x=filtered_data['Date'], y=filtered_data['Open'], mode='lines+markers', name='Open Price', hovertext=hover_text_open, hoverinfo='text'), row=2, col=1)

# Update layout
fig.update_layout(title_text="Investment Value and Open Price Over Time", height=800, showlegend=False,
                  hovermode="x unified")  # Unified hover mode for both subplots

# # Define the position of the vertical line
x_value = filtered_data["Date"].min().strftime('%Y-%m-%d')

# Add a vertical line shape to highlight the hovered date on both subplots
fig.add_shape(type="line", x0=x_value, y0=0, x1=x_value, y1=1, row=1, col=1,
              line=dict(color="RoyalBlue", width=1, dash="dash"))

fig.add_shape(type="line", x0=x_value, y0=0, x1=x_value, y1=1, row=2, col=1,
              line=dict(color="RoyalBlue", width=1, dash="dash"))

# Show plot
fig.show()
